In [1]:

# conda remove --name SCENIC --all
# conda create --name SCENIC python=3.6.3 
# conda activate SCENIC
# python -m pip install pandas
# git clone https://github.com/aertslab/pycisTopic.git
# cd pycisTopic
# python -m pip install .
# python -m pip install --upgrade pip setuptools
# python -m pip install scanpy
# python -m pip install  pyranges
# python -m pip install requests --ignore-installed certifi
# python -m pip install ray
# python -m pip install tmtoolkit
# python -m pip install gensim
# python -m pip install loomxpy
# python -m pip install pyrle
# python -m pip install polars
# python -m pip  install typing-extensions --upgrade
# python -m pip install pyarrow
# python -m pip install loompy
# python -m pip install pyscenic
# python -m pip install pickle5
# python -m pip install ipykernel
#  python -m pip install pyBigWig
# install jupyter-lab
# mv pycisTopic/ pycisTopic_repo
#git clone https://github.com/aertslab/scenicplus
#cd scenicplus
#pip install .



import warnings
warnings.simplefilter(action='ignore')
import pandas as pd
import scanpy
import os
import requests
import pyranges as pr
import pycisTopic
from pycisTopic.pseudobulk_peak_calling import export_pseudobulk
from pycisTopic.iterative_peak_calling import *
import anndata
import pickle
import pyarrow 
import re


def CountFrequency(my_list):
 
    # Creating an empty dictionary
    freq = {}
    for items in my_list:
        freq[items] = my_list.count(items)
 
    for key, value in freq.items():
        print("% s : % d" % (key, value))
 

outDir="SCENIC_results/"
tmpDir="/tmp/"
#ray.shutdown()


In [ ]:
import pybiomart as pbm
import pandas as pd

dataset_zfish = pbm.Dataset(name='drerio_gene_ensembl',  host='http://www.ensembl.org')
dataset_human = pbm.Dataset(name='hsapiens_gene_ensembl',  host='http://www.ensembl.org')

# run this to create the motifs_annotation for zebra fish
#https://github.com/JoGraesslin/Zebrafish_SCENIC
# wget https://www.ensembl.org/biomart/martview/c4dfb7b0b8ff5effe4c0bd97e74eb8d0

# wget https://resources.aertslab.org/cistarget/motif_collections/v10nr_clust_public/snapshots/motifs-v10-nr.hgnc-m0.00001-o0.0.tbl
aerts_tbl=pd.read_csv("/home/lper0012/tasks/margo.montandon/reference/motifs-v10-nr.hgnc-m0.00001-o0.0.tbl",sep="\t") #human .tbl file

## these tables were extracted from the reults section of  after selecting the specific species https://www.ensembl.org/biomart/martview/
## select the especies and in the attributes section selesc GENE ->Gene stable ID EXTERNAL->HGNC/ZFIN
ensembl_human=pd.read_csv("human2zfish/ens_to_human.txt",sep="\t") #contains Ensembl Gene stable ID and HGNC symbol
ensembl_zfish=pd.read_csv("human2zfish/ens_to_zfin.txt",sep="\t") #contains Ensembl Zebrafish gene stable ID and ZFIN symbol


In [ ]:


# https://www.alliancegenome.org/downloads#orthology
alliance_human_to_zfish=pd.read_csv("human2zfish/ORTHOLOGY-ALLIANCE_COMBINED.tsv",skiprows=15,sep="\t") #contains orthology symbols from human and zfin


# create oma file from downloading the human and the fish stanalone datasets from here: https://omabrowser.org/oma/export/
# then run OMA within the folder with  OutgroupSpecies := 'none'; 
# in folder /OMA.2.6.0/Output/PairwiseOrthologs
oma_human_to_zfish=pd.read_csv("human2zfish/OMA.2.6.0/Output/PairwiseOrthologs/DANRE-HUMAN.txt",skiprows=4,sep="\t",header=None) #contains orthology ENSG and ENSDARG from oma orthology browser
#oma_human_to_zfish.columns=["Protein 1","Protein 2","Protein ID1","Protein ID2","Orthology type","OMA group"]
oma_human_to_zfish=oma_human_to_zfish.rename(columns={0:'chr',1:'position',2:'ens_zf',3:'ens_hg',4:'n_orth',5:'score'})

oma_human_to_zfish


In [ ]:

# get orthologuoes IDs
# https://www.youtube.com/watch?v=EwfOQsr2O-M 
ensembl_human_to_zfish=pd.read_csv("human2zfish/human_to_zfin.txt",sep="\t") #contains orthology ENSG and ENSDARG from ensembl 


In [ ]:

#create an ensembl orthologue table
human_zf = ensembl_human.merge(ensembl_human_to_zfish, on='Gene stable ID', how='left')
human_zf

In [ ]:

ensembl_zfish=ensembl_zfish.rename(columns={'Gene stable ID':'Zebrafish gene stable ID'})
ensembl_zfish

In [ ]:

zf_zf = ensembl_zfish.merge(ensembl_human_to_zfish, on='Zebrafish gene stable ID', how='left')
zf_zf

In [ ]:

human_zf_grouped = human_zf.groupby('HGNC symbol').agg({'Gene stable ID': lambda x: tuple(x), 'Zebrafish gene stable ID': lambda x: tuple(x)})
human_zf_grouped

In [ ]:
human_zf_grouped["hg_symbol"]=human_zf_grouped.index


In [ ]:

zf_zf_grouped = zf_zf.groupby('ZFIN symbol').agg({'Gene stable ID': lambda x: tuple(x), 'Zebrafish gene stable ID': lambda x: tuple(x)})
zf_zf_grouped["zf_symbol"]=zf_zf_grouped.index
human_zf_mapping = human_zf_grouped.merge(zf_zf_grouped, on='Gene stable ID',how="left")
ens_human_zf_mapping=human_zf_mapping.dropna(subset=['hg_symbol', 'zf_symbol'])

In [ ]:

#create an oma orthology table
oma_human_to_zfish

In [ ]:
oma_human_to_zfish=oma_human_to_zfish.merge(ensembl_human,left_on="ens_hg",right_on='Gene stable ID')
oma_human_to_zfish=oma_human_to_zfish.merge(ensembl_zfish,left_on="ens_zf",right_on='Zebrafish gene stable ID')
oma_human_to_zfish=oma_human_to_zfish.dropna(subset=['HGNC symbol', 'ZFIN symbol'])
oma_human_to_zfish=oma_human_to_zfish.rename(columns={'HGNC symbol':'hg_symbol','ZFIN symbol':'zf_symbol'})
oma_human_to_zfish

In [ ]:

#prepare alliance db
alliance_human_to_zfish=alliance_human_to_zfish[alliance_human_to_zfish["Gene1SpeciesName"]=="Homo sapiens"] 
alliance_human_to_zfish=alliance_human_to_zfish[alliance_human_to_zfish["Gene2SpeciesName"]=="Danio rerio"]
alliance_human_to_zfish=alliance_human_to_zfish.rename(columns={'Gene1Symbol':'hg_symbol','Gene2Symbol':'zf_symbol'})

In [ ]:
#combine orthologies
gene_orthologies=pd.concat([ens_human_zf_mapping.iloc[:,[2,4]],alliance_human_to_zfish.iloc[:,[1,5]],oma_human_to_zfish.iloc[:,[5,7]]])
#gene_orthologies=pd.concat([ens_human_zf_mapping.iloc[:,[2,4]],alliance_human_to_zfish.iloc[:,[1,5]]])
gene_orthologies=gene_orthologies.drop_duplicates()
#change gene_names
aerts_tbl=aerts_tbl.merge(gene_orthologies,left_on="gene_name",right_on='hg_symbol')
aerts_tbl['gene_name']=aerts_tbl['zf_symbol']
aerts_tbl=aerts_tbl.drop(columns=['hg_symbol', 'zf_symbol'])
#save final file
aerts_tbl.to_csv("motifs-v10-zfish.tbl",index=False,sep ='\t')

In [ ]:
######################################## updated motifs files ######################################## ######################################## 

In [2]:
import pybiomart as pbm
import pandas as pd

dataset_zfish = pbm.Dataset(name='drerio_gene_ensembl',  host='http://www.ensembl.org')
dataset_human = pbm.Dataset(name='hsapiens_gene_ensembl',  host='http://www.ensembl.org')
dataset_mouse = pbm.Dataset(name='mmusculus_gene_ensembl',  host='http://www.ensembl.org')


In [3]:

dataset_mouse

<biomart.Dataset name='mmusculus_gene_ensembl', display_name=''>

In [4]:

# run this to create the motifs_annotation for zebra fish
# https://github.com/JoGraesslin/Zebrafish_SCENIC
# wget https://www.ensembl.org/biomart/martview/c4dfb7b0b8ff5effe4c0bd97e74eb8d0
ensembl_human=pd.read_csv("human2zfish/ens_to_human.txt",sep="\t") #contains Ensembl Gene stable ID and HGNC symbol
ensembl_zfish=pd.read_csv("human2zfish/ens_to_zfin.txt",sep="\t") #contains Ensembl Zebrafish gene stable ID and ZFIN symbol



In [19]:

# wget https://resources.aertslab.org/cistarget/motif_collections/v10nr_clust_public/snapshots/motifs-v10-nr.hgnc-m0.00001-o0.0.tbl
#aerts_tbl=pd.read_csv("/home/lper0012/tasks/margo.montandon/reference/motifs-v10-nr.hgnc-m0.00001-o0.0.tbl",sep="\t") #human .tbl file
aerts_human_tbl=pd.read_csv("/home/lper0012/tasks/margo.montandon/reference/v10nr_clust_public/snapshots/motifs-v10-nr.hgnc-m0.00001-o0.0.tbl",sep="\t")
aerts_mouse_tbl=pd.read_csv("/home/lper0012/tasks/margo.montandon/reference/v10nr_clust_public/snapshots/motifs-v10-nr.mgi-m0.00001-o0.0.tbl",sep="\t")
aerts_chicken_tbl=pd.read_csv("/home/lper0012/tasks/margo.montandon/reference/v10nr_clust_public/snapshots/motifs-v10-nr.chicken-m0.00001-o0.0.tbl",sep="\t")
aerts_fly_tbl=pd.read_csv("/home/lper0012/tasks/margo.montandon/reference/v10nr_clust_public/snapshots/motifs-v10-nr.flybase-m0.00001-o0.0.tbl",sep="\t")


In [20]:

frames = [aerts_human_tbl, aerts_mouse_tbl, aerts_chicken_tbl,aerts_fly_tbl]
aerts_all_tbl = pd.concat(frames)

In [6]:
searchfor=['met','cdh15','pax7a','wnt2','notch3','gfap']

searchfor=['met','cdh15','pax7a','wnt2','notch3','gfap','boc','cdon','dhh','gas1a','gas1b','gli1','gli2a','gli2b','gli3','hhip','hhipl1','hhipl2','ihha',
'ihhb','kif7','mir214a','ptch1','ptch2','scube2','shha','shhb','si:ch211-136a13.1','smo','sufu']

#CLUSTERS	MOST IMPORTANT GENES		OTHER GENES
#Satellite cells	pax3a+ / met+	co-expression	cdh15, pax7a, wnt2, notch3
#External cell layer	pax3a+ / meox1+	co-expression	notch3
#Commited myoblasts	myod1+ and myog+		
#Spinal cord (not interesting for us)	gfap+		
#Neural crest  (also not interesting for us)	sox10+	

In [7]:
aerts_human_tbl.gene_name[aerts_human_tbl.gene_name.str.contains(('|'.join(searchfor)), na=False,case=False)]

#DA11; HGFR; AUTS9; RCCP2; c-Met; DFNB97
#motifs_ort.Direct_annot[motifs_ort.Direct_annot.str.contains(r"met", na=False)]


7432        GLI1
7434        GLI3
7435        GLI1
7437        GLI3
7438        GLI1
           ...  
243912      GLI3
247261      GLI1
249197      GLI3
249198      GLI3
252974    METTL3
Name: gene_name, Length: 141, dtype: object

In [8]:
aerts_mouse_tbl.gene_name[aerts_mouse_tbl.gene_name.str.contains(('|'.join(searchfor)), na=False,case=False)]


7421        Gli1
7423        Gli3
7424        Gli1
7426        Gli3
7427        Gli1
           ...  
243105      Gli3
246425      Gli1
248326      Gli3
248327      Gli3
252004    Mettl3
Name: gene_name, Length: 141, dtype: object

In [8]:
aerts_chicken_tbl.gene_name[aerts_chicken_tbl.gene_name.str.contains(('|'.join(searchfor)), na=False,case=False)]

10541     METTL14
116275    METTL14
116279    METTL14
179227    METTL14
186829     METTL3
Name: gene_name, dtype: object

In [9]:
aerts_fly_tbl[aerts_fly_tbl.gene_name.str.contains(('|'.join(searchfor)), na=False,case=False)]


,#motif_id,motif_name,motif_description,source_name,source_version,gene_name,motif_similarity_qvalue,similar_motif_id,similar_motif_description,orthologous_identity,orthologous_gene_name,orthologous_species,description
194,cisbp__M00011,M00011,TYE7[gene ID: YOR344C species: Saccharomyces c...,cisbp,2.00,Met,7.290160e-06,metacluster_57.3,V$ARNT_02: Arnt,0.106145,ENSMUSG00000015522,M. musculus,gene is orthologous to ENSMUSG00000015522 in M...
291,metacluster_128.2,M00119,Bhlhe40[gene ID: ENSMUSG00000030103 species: M...,cisbp,2.00,Met,3.181990e-06,metacluster_14.6,V$BMAL1CLOCK_01: BMAL1:CLOCK,0.132588,ENSG00000133794,H. sapiens,motif similar to transfac_pro__M08909 ('V$BMAL...
297,metacluster_13.3,M00121,Max[gene ID: ENSMUSG00000059436 species: Mus m...,cisbp,2.00,Met,4.122990e-06,metacluster_13.3,Clock[gene ID: ENSMUSG00000029238 species: Mus...,0.122905,ENSMUSG00000029238,M. musculus,gene is orthologous to ENSMUSG00000029238 in M...
302,metacluster_13.3,M00122,Max[gene ID: ENSMUSG00000059436 species: Mus m...,cisbp,2.00,Met,2.038460e-06,metacluster_13.3,Clock[gene ID: ENSMUSG00000029238 species: Mus...,0.122905,ENSMUSG00000029238,M. musculus,gene is orthologous to ENSMUSG00000029238 in M...
1242,cisbp__M00313,M00313,Gsc2[gene ID: ENSMUSG00000022738 species: Mus ...,cisbp,2.00,Met,2.253010e-09,taipale_tf_pairs__ARNTL_PITX1_NNCACGTGNNNNRGMT...,ARNTL [bHLH] - PITX1 [Homeo],0.132588,ENSG00000133794,H. sapiens,motif similar to taipale_tf_pairs__ARNTL_PITX1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159785,transfac_public__M00237,M00237,V$AHRARNT_02: AhR:Arnt,transfac_public,7.0,Met,0.000000e+00,None,None,0.106145,ENSMUSG00000015522,M. musculus,gene is orthologous to ENSMUSG00000015522 in M...
160050,metacluster_10.3,M00434,P$PIF3_01: PIF3,transfac_public,7.0,Met,9.732900e-06,flyfactorsurvey__Met_Clk_SANGER_5_FBgn0002723,Met_Clk_SANGER_5_FBgn0002723,1.000000,None,None,motif similar to flyfactorsurvey__Met_Clk_SANG...
160192,metacluster_57.3,M00539,V$ARNT_02: Arnt,transfac_public,7.0,Met,0.000000e+00,None,None,0.106145,ENSMUSG00000015522,M. musculus,gene is orthologous to ENSMUSG00000015522 in M...
160225,metacluster_198.2,M00615,V$MYCMAX_03: c-Myc:Max,transfac_public,7.0,Met,5.135080e-13,metacluster_57.3,V$ARNT_02: Arnt,0.106145,ENSMUSG00000015522,M. musculus,gene is orthologous to ENSMUSG00000015522 in M...


In [12]:
aerts_fly_tbl[aerts_fly_tbl.gene_name.str.contains(r'Met')]

,#motif_id,motif_name,motif_description,source_name,source_version,gene_name,motif_similarity_qvalue,similar_motif_id,similar_motif_description,orthologous_identity,orthologous_gene_name,orthologous_species,description
194,cisbp__M00011,M00011,TYE7[gene ID: YOR344C species: Saccharomyces c...,cisbp,2.00,Met,7.290160e-06,metacluster_57.3,V$ARNT_02: Arnt,0.106145,ENSMUSG00000015522,M. musculus,gene is orthologous to ENSMUSG00000015522 in M...
291,metacluster_128.2,M00119,Bhlhe40[gene ID: ENSMUSG00000030103 species: M...,cisbp,2.00,Met,3.181990e-06,metacluster_14.6,V$BMAL1CLOCK_01: BMAL1:CLOCK,0.132588,ENSG00000133794,H. sapiens,motif similar to transfac_pro__M08909 ('V$BMAL...
297,metacluster_13.3,M00121,Max[gene ID: ENSMUSG00000059436 species: Mus m...,cisbp,2.00,Met,4.122990e-06,metacluster_13.3,Clock[gene ID: ENSMUSG00000029238 species: Mus...,0.122905,ENSMUSG00000029238,M. musculus,gene is orthologous to ENSMUSG00000029238 in M...
302,metacluster_13.3,M00122,Max[gene ID: ENSMUSG00000059436 species: Mus m...,cisbp,2.00,Met,2.038460e-06,metacluster_13.3,Clock[gene ID: ENSMUSG00000029238 species: Mus...,0.122905,ENSMUSG00000029238,M. musculus,gene is orthologous to ENSMUSG00000029238 in M...
1242,cisbp__M00313,M00313,Gsc2[gene ID: ENSMUSG00000022738 species: Mus ...,cisbp,2.00,Met,2.253010e-09,taipale_tf_pairs__ARNTL_PITX1_NNCACGTGNNNNRGMT...,ARNTL [bHLH] - PITX1 [Homeo],0.132588,ENSG00000133794,H. sapiens,motif similar to taipale_tf_pairs__ARNTL_PITX1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159785,transfac_public__M00237,M00237,V$AHRARNT_02: AhR:Arnt,transfac_public,7.0,Met,0.000000e+00,None,None,0.106145,ENSMUSG00000015522,M. musculus,gene is orthologous to ENSMUSG00000015522 in M...
160050,metacluster_10.3,M00434,P$PIF3_01: PIF3,transfac_public,7.0,Met,9.732900e-06,flyfactorsurvey__Met_Clk_SANGER_5_FBgn0002723,Met_Clk_SANGER_5_FBgn0002723,1.000000,None,None,motif similar to flyfactorsurvey__Met_Clk_SANG...
160192,metacluster_57.3,M00539,V$ARNT_02: Arnt,transfac_public,7.0,Met,0.000000e+00,None,None,0.106145,ENSMUSG00000015522,M. musculus,gene is orthologous to ENSMUSG00000015522 in M...
160225,metacluster_198.2,M00615,V$MYCMAX_03: c-Myc:Max,transfac_public,7.0,Met,5.135080e-13,metacluster_57.3,V$ARNT_02: Arnt,0.106145,ENSMUSG00000015522,M. musculus,gene is orthologous to ENSMUSG00000015522 in M...


In [10]:
type(aerts_human_tbl)

aerts_human_tbl = pd.concat([aerts_human_tbl,aerts_fly_tbl[aerts_fly_tbl.gene_name.str.contains(r'Met')]], axis=0)


In [11]:
aerts_human_tbl[aerts_human_tbl.gene_name.str.contains((r'Met'), na=False)]


,#motif_id,motif_name,motif_description,source_name,source_version,gene_name,motif_similarity_qvalue,similar_motif_id,similar_motif_description,orthologous_identity,orthologous_gene_name,orthologous_species,description
194,cisbp__M00011,M00011,TYE7[gene ID: YOR344C species: Saccharomyces c...,cisbp,2.00,Met,7.290160e-06,metacluster_57.3,V$ARNT_02: Arnt,0.106145,ENSMUSG00000015522,M. musculus,gene is orthologous to ENSMUSG00000015522 in M...
291,metacluster_128.2,M00119,Bhlhe40[gene ID: ENSMUSG00000030103 species: M...,cisbp,2.00,Met,3.181990e-06,metacluster_14.6,V$BMAL1CLOCK_01: BMAL1:CLOCK,0.132588,ENSG00000133794,H. sapiens,motif similar to transfac_pro__M08909 ('V$BMAL...
297,metacluster_13.3,M00121,Max[gene ID: ENSMUSG00000059436 species: Mus m...,cisbp,2.00,Met,4.122990e-06,metacluster_13.3,Clock[gene ID: ENSMUSG00000029238 species: Mus...,0.122905,ENSMUSG00000029238,M. musculus,gene is orthologous to ENSMUSG00000029238 in M...
302,metacluster_13.3,M00122,Max[gene ID: ENSMUSG00000059436 species: Mus m...,cisbp,2.00,Met,2.038460e-06,metacluster_13.3,Clock[gene ID: ENSMUSG00000029238 species: Mus...,0.122905,ENSMUSG00000029238,M. musculus,gene is orthologous to ENSMUSG00000029238 in M...
1242,cisbp__M00313,M00313,Gsc2[gene ID: ENSMUSG00000022738 species: Mus ...,cisbp,2.00,Met,2.253010e-09,taipale_tf_pairs__ARNTL_PITX1_NNCACGTGNNNNRGMT...,ARNTL [bHLH] - PITX1 [Homeo],0.132588,ENSG00000133794,H. sapiens,motif similar to taipale_tf_pairs__ARNTL_PITX1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159785,transfac_public__M00237,M00237,V$AHRARNT_02: AhR:Arnt,transfac_public,7.0,Met,0.000000e+00,None,None,0.106145,ENSMUSG00000015522,M. musculus,gene is orthologous to ENSMUSG00000015522 in M...
160050,metacluster_10.3,M00434,P$PIF3_01: PIF3,transfac_public,7.0,Met,9.732900e-06,flyfactorsurvey__Met_Clk_SANGER_5_FBgn0002723,Met_Clk_SANGER_5_FBgn0002723,1.000000,None,None,motif similar to flyfactorsurvey__Met_Clk_SANG...
160192,metacluster_57.3,M00539,V$ARNT_02: Arnt,transfac_public,7.0,Met,0.000000e+00,None,None,0.106145,ENSMUSG00000015522,M. musculus,gene is orthologous to ENSMUSG00000015522 in M...
160225,metacluster_198.2,M00615,V$MYCMAX_03: c-Myc:Max,transfac_public,7.0,Met,5.135080e-13,metacluster_57.3,V$ARNT_02: Arnt,0.106145,ENSMUSG00000015522,M. musculus,gene is orthologous to ENSMUSG00000015522 in M...


In [13]:


# https://www.alliancegenome.org/downloads#orthology
alliance_human_to_zfish=pd.read_csv("human2zfish/ORTHOLOGY-ALLIANCE_COMBINED.tsv",skiprows=15,sep="\t") #contains orthology symbols from human and zfin



In [14]:

alliance_human_to_zfish

,Gene1ID,Gene1Symbol,Gene1SpeciesTaxonID,Gene1SpeciesName,Gene2ID,Gene2Symbol,Gene2SpeciesTaxonID,Gene2SpeciesName,Algorithms,AlgorithmsMatch,OutOfAlgorithms,IsBestScore,IsBestRevScore
0,Xenbase:XB-GENE-5764740,cyp2a6.7,NCBITaxon:8364,Xenopus tropicalis,SGD:S000002810,DIT2,NCBITaxon:559292,Saccharomyces cerevisiae,Ensembl Compara|InParanoid|SonicParanoid,3,9,Yes,Yes
1,Xenbase:XB-GENE-5831571,cyp2c8.1,NCBITaxon:8364,Xenopus tropicalis,SGD:S000002810,DIT2,NCBITaxon:559292,Saccharomyces cerevisiae,Ensembl Compara|InParanoid|SonicParanoid,3,9,Yes,Yes
2,Xenbase:XB-GENE-1007290,cyp2u1,NCBITaxon:8364,Xenopus tropicalis,SGD:S000002810,DIT2,NCBITaxon:559292,Saccharomyces cerevisiae,Ensembl Compara|InParanoid|SonicParanoid,3,9,Yes,Yes
3,Xenbase:XB-GENE-5903984,cyp2a6.3,NCBITaxon:8364,Xenopus tropicalis,SGD:S000002810,DIT2,NCBITaxon:559292,Saccharomyces cerevisiae,SonicParanoid|Ensembl Compara|InParanoid,3,9,Yes,Yes
4,Xenbase:XB-GENE-5903934,cyp2a6,NCBITaxon:8364,Xenopus tropicalis,SGD:S000002810,DIT2,NCBITaxon:559292,Saccharomyces cerevisiae,Ensembl Compara|InParanoid|SonicParanoid,3,9,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
914731,HGNC:26959,WDR97,NCBITaxon:9606,Homo sapiens,MGI:5594498,Wdr97,NCBITaxon:10090,Mus musculus,OrthoInspector|OMA|OrthoFinder|Hieranoid|HGNC|...,9,10,Yes,Yes
914732,ZFIN:ZDB-GENE-160728-123,si:ch73-174h16.5,NCBITaxon:7955,Danio rerio,MGI:5594498,Wdr97,NCBITaxon:10090,Mus musculus,OrthoInspector|OrthoFinder|Hieranoid|SonicPara...,8,10,Yes,Yes
914733,RGD:1562063,Wdr97,NCBITaxon:10116,Rattus norvegicus,MGI:5594498,Wdr97,NCBITaxon:10090,Mus musculus,OrthoInspector|OMA|OrthoFinder|SonicParanoid|H...,9,9,Yes,Yes
914734,HGNC:44570,C3orf86P,NCBITaxon:9606,Homo sapiens,MGI:5594708,Gm35549,NCBITaxon:10090,Mus musculus,OrthoInspector|OrthoFinder|Hieranoid|HGNC|Soni...,6,10,Yes,Yes


In [15]:

# get orthologuoes IDs
# https://www.youtube.com/watch?v=EwfOQsr2O-M 
ensembl_human_to_zfish=pd.read_csv("human2zfish/human_to_zfin.txt",sep="\t") #contains orthology ENSG and ENSDARG from ensembl 


In [26]:


#create an ensembl orthologue table
human_zf = ensembl_human.merge(ensembl_human_to_zfish, on='Gene stable ID', how='left')
human_zf

,Gene stable ID,Gene stable ID version_x,HGNC ID,HGNC symbol,Gene stable ID version_y,Zebrafish gene stable ID,Zebrafish gene name
0,ENSG00000210049,ENSG00000210049.1,HGNC:7481,MT-TF,ENSG00000210049.1,NaN,NaN
1,ENSG00000211459,ENSG00000211459.2,HGNC:7470,MT-RNR1,ENSG00000211459.2,NaN,NaN
2,ENSG00000210077,ENSG00000210077.1,HGNC:7500,MT-TV,ENSG00000210077.1,NaN,NaN
3,ENSG00000210082,ENSG00000210082.2,HGNC:7471,MT-RNR2,ENSG00000210082.2,NaN,NaN
4,ENSG00000209082,ENSG00000209082.1,HGNC:7490,MT-TL1,ENSG00000209082.1,NaN,NaN
...,...,...,...,...,...,...,...
77454,ENSG00000236500,ENSG00000236500.1,HGNC:1646,CD24P1,ENSG00000236500.1,NaN,NaN
77455,ENSG00000197312,ENSG00000197312.13,HGNC:24578,DDI2,ENSG00000197312.13,ENSDARG00000006477,ddi2
77456,ENSG00000215695,ENSG00000215695.2,HGNC:10458,RSC1A1,ENSG00000215695.2,NaN,NaN
77457,ENSG00000271742,ENSG00000271742.1,NaN,NaN,ENSG00000271742.1,NaN,NaN


In [27]:

ensembl_zfish=ensembl_zfish.rename(columns={'Gene stable ID':'Zebrafish gene stable ID'})


In [28]:
ensembl_zfish[ensembl_zfish['ZFIN symbol'].str.contains(r"met", na=False)]


,Zebrafish gene stable ID,Gene stable ID version,ZFIN ID,ZFIN symbol
345,ENSDARG00000068893,ENSDARG00000068893.6,ZDB-GENE-041010-21,mettl5
925,ENSDARG00000102571,ENSDARG00000102571.2,ZDB-GENE-030131-2172,metap2b
1056,ENSDARG00000089236,ENSDARG00000089236.4,ZDB-GENE-130530-958,mettl15
1407,ENSDARG00000012827,ENSDARG00000012827.8,ZDB-GENE-030131-9498,mettl3
3400,ENSDARG00000060889,ENSDARG00000060889.7,ZDB-GENE-120215-64,mettl7a.3
3447,ENSDARG00000056726,ENSDARG00000056726.8,ZDB-GENE-061027-239,mettl7a.1
5072,ENSDARG00000069507,ENSDARG00000069507.6,ZDB-GENE-060421-5918,mettl27
6203,ENSDARG00000010905,ENSDARG00000010905.11,ZDB-GENE-060929-60,mettl13
8462,ENSDARG00000016343,ENSDARG00000016343.10,ZDB-GENE-040718-53,mettl21e
8623,ENSDARG00000055437,ENSDARG00000055437.8,ZDB-GENE-080204-102,mettl21cb


In [29]:

ensembl_human['HGNC symbol'][ensembl_human['Gene stable ID'].str.contains(r"ENSG00000133794", na=False)]='MET'



# this ID ENSG00000133794 I saw in the fly data as an orthologoues of MET in humans
#	Gene stable ID	Gene stable ID version	HGNC ID	HGNC symbol
#43998	ENSG00000133794	ENSG00000133794.20	HGNC:701	BMAL1



In [30]:
ensembl_human[ensembl_human['Gene stable ID'].str.contains(r"ENSG00000133794", na=False)]

,Gene stable ID,Gene stable ID version,HGNC ID,HGNC symbol
43998,ENSG00000133794,ENSG00000133794.20,HGNC:701,MET


In [31]:

zf_zf = ensembl_zfish.merge(ensembl_human_to_zfish, on='Zebrafish gene stable ID', how='left')
zf_zf

,Zebrafish gene stable ID,Gene stable ID version_x,ZFIN ID,ZFIN symbol,Gene stable ID,Gene stable ID version_y,Zebrafish gene name
0,ENSDARG00000097503,ENSDARG00000097503.3,NaN,NaN,NaN,NaN,NaN
1,ENSDARG00000104512,ENSDARG00000104512.2,NaN,NaN,NaN,NaN,NaN
2,ENSDARG00000089622,ENSDARG00000089622.4,NaN,NaN,ENSG00000154783,ENSG00000154783.12,NaN
3,ENSDARG00000093926,ENSDARG00000093926.3,NaN,NaN,NaN,NaN,NaN
4,ENSDARG00000093957,ENSDARG00000093957.2,ZDB-GENE-050506-102,si:dkey-251i10.2,NaN,NaN,NaN
...,...,...,...,...,...,...,...
41659,ENSDARG00000117669,ENSDARG00000117669.1,NaN,NaN,NaN,NaN,NaN
41660,ENSDARG00000117232,ENSDARG00000117232.1,NaN,NaN,NaN,NaN,NaN
41661,ENSDARG00000117768,ENSDARG00000117768.1,NaN,NaN,NaN,NaN,NaN
41662,ENSDARG00000117581,ENSDARG00000117581.1,NaN,NaN,NaN,NaN,NaN


In [32]:

human_zf_grouped = human_zf.groupby('HGNC symbol').agg({'Gene stable ID': lambda x: tuple(x), 'Zebrafish gene stable ID': lambda x: tuple(x)})
human_zf_grouped

,Gene stable ID,Zebrafish gene stable ID
HGNC symbol,,
A1BG,"(ENSG00000121410,)","(nan,)"
A1BG-AS1,"(ENSG00000268895,)","(nan,)"
A1CF,"(ENSG00000148584,)","(ENSDARG00000002968,)"
A2M,"(ENSG00000175899,)","(nan,)"
A2M-AS1,"(ENSG00000245105,)","(nan,)"
...,...,...
ZYG11B,"(ENSG00000162378, ENSG00000162378)","(ENSDARG00000007737, ENSDARG00000101799)"
ZYX,"(ENSG00000285443, ENSG00000159840)","(nan, ENSDARG00000075733)"
ZYXP1,"(ENSG00000274572,)","(nan,)"


In [33]:
human_zf_grouped[ human_zf_grouped.index.str.contains(r'MET')]

,Gene stable ID,Zebrafish gene stable ID
HGNC symbol,,
COMETT,"(ENSG00000231210,)","(nan,)"
MET,"(ENSG00000133794, ENSG00000133794, ENSG0000010...","(ENSDARG00000035732, ENSDARG00000006791, ENSDA..."
METAP1,"(ENSG00000164024,)","(ENSDARG00000033440,)"
METAP1D,"(ENSG00000292097, ENSG00000172878)","(nan, ENSDARG00000019715)"
METAP2,"(ENSG00000111142,)","(ENSDARG00000102571,)"
METAP2P1,"(ENSG00000234119,)","(nan,)"
METRN,"(ENSG00000103260,)","(ENSDARG00000030367,)"
METRNL,"(ENSG00000275031, ENSG00000176845, ENSG0000017...","(nan, ENSDARG00000101914, ENSDARG00000007289)"
METTL1,"(ENSG00000037897,)","(ENSDARG00000076518,)"


In [34]:
human_zf_grouped["hg_symbol"]=human_zf_grouped.index


In [35]:

zf_zf_grouped = zf_zf.groupby('ZFIN symbol').agg({'Gene stable ID': lambda x: tuple(x), 'Zebrafish gene stable ID': lambda x: tuple(x)})
zf_zf_grouped["zf_symbol"]=zf_zf_grouped.index
human_zf_mapping = human_zf_grouped.merge(zf_zf_grouped, on='Gene stable ID',how="left")


In [36]:

human_zf_mapping[human_zf_mapping['hg_symbol'].str.contains(r'MET')]

,Gene stable ID,Zebrafish gene stable ID_x,hg_symbol,Zebrafish gene stable ID_y,zf_symbol
5076,"(ENSG00000231210,)","(nan,)",COMETT,NaN,NaN
17539,"(ENSG00000133794, ENSG00000133794, ENSG0000010...","(ENSDARG00000035732, ENSDARG00000006791, ENSDA...",MET,NaN,NaN
17540,"(ENSG00000164024,)","(ENSDARG00000033440,)",METAP1,NaN,NaN
17541,"(ENSG00000292097, ENSG00000172878)","(nan, ENSDARG00000019715)",METAP1D,NaN,NaN
17542,"(ENSG00000111142,)","(ENSDARG00000102571,)",METAP2,"(ENSDARG00000102571,)",metap2b
17543,"(ENSG00000234119,)","(nan,)",METAP2P1,NaN,NaN
17544,"(ENSG00000103260,)","(ENSDARG00000030367,)",METRN,NaN,NaN
17545,"(ENSG00000275031, ENSG00000176845, ENSG0000017...","(nan, ENSDARG00000101914, ENSDARG00000007289)",METRNL,NaN,NaN
17546,"(ENSG00000037897,)","(ENSDARG00000076518,)",METTL1,"(ENSDARG00000076518,)",mettl1
17547,"(ENSG00000010165,)","(ENSDARG00000010905,)",METTL13,"(ENSDARG00000010905,)",mettl13


In [37]:
ens_human_zf_mapping=human_zf_mapping.dropna(subset=['hg_symbol', 'zf_symbol'])


In [38]:

# create oma file from downloading the human and the fish stanalone datasets from here: https://omabrowser.org/oma/export/
# then run OMA within the folder with  OutgroupSpecies := 'none'; 
# in folder /OMA.2.6.0/Output/PairwiseOrthologs
oma_human_to_zfish=pd.read_csv("human2zfish/OMA.2.6.0/Output/PairwiseOrthologs/DANRE-HUMAN.txt",skiprows=4,sep="\t",header=None) #contains orthology ENSG and ENSDARG from oma orthology browser
#oma_human_to_zfish.columns=["Protein 1","Protein 2","Protein ID1","Protein ID2","Orthology type","OMA group"]
oma_human_to_zfish=oma_human_to_zfish.rename(columns={0:'chr',1:'position',2:'ens_zf',3:'ens_hg',4:'n_orth',5:'score'})


In [39]:

(oma_human_to_zfish[oma_human_to_zfish.ens_hg.str.contains(r'MET')])


,chr,position,ens_zf,ens_hg,n_orth,score
38106,9943,88007,DANRE009943 | ENSDARP00000146656.1 | ENSDART00...,HUMAN088007 | ENSP00000383185.3 | ENST00000400...,1:1,4730.0
39620,12838,3300,DANRE012838 | ENSDARP00000148805.1 | ENSDART00...,HUMAN003300 | ENSP00000455755.1 | ENST00000494...,1:many,NaN
60640,27540,90949,DANRE027540 | ENSDARP00000123904.1 | ENSDART00...,HUMAN090949 | ENSP00000317272.6 | ENST00000318...,1:1,1653.0


In [40]:

oma_human_to_zfish.ens_hg.str.split("|",expand=True)


,0,1,2,3,4,5,6
0,HUMAN028095,ENSP00000378288.4,ENST00000394809.9,ENSG00000140795.13,HOG:D0616192.1a,myosin light chain kinase 3 [Source:HGNC Symb...,None
1,HUMAN096889,ENSP00000367880.3,ENST00000378617.4,ENSG00000165282.14,HOG:D0659026.1b.4a,phosphatidylinositol glycan anchor biosynthes...,None
2,HUMAN059690,ENSP00000248933.6,ENST00000248933.11,ENSG00000100095.19,HOG:D0629707.1a,seizure related 6 homolog like [Source:HGNC S...,None
3,HUMAN030660,ENSP00000301335.5,ENST00000301335.10,ENSG00000167703.15,HOG:D0684981.2a.3b.4a.11c,solute carrier family 43 member 2 [Source:HGN...,None
4,HUMAN035462,ENSP00000380888.2,ENST00000397786.7,ENSG00000108510.10,A0A024QZ75,HOG:D0659262.2b.6b.2a,mediator complex subunit 13 [Source:HGNC Symb...
...,...,...,...,...,...,...,...
85279,HUMAN011378,ENSP00000261167.2,ENST00000261167.7,ENSG00000084463.8,A0A024RAW9; F5GXS9,HOG:D0673655.1c.6a.1a.1c.4h,WW domain binding protein 11 [Source:HGNC Sym...
85280,HUMAN080350,ENSP00000250535.4,ENST00000250535.5,ENSG00000129596.5,HOG:D0690063.3b.9a,cysteine dioxygenase type 1 [Source:HGNC Symb...,None
85281,HUMAN022103,ENSP00000380872.2,ENST00000397766.4,ENSG00000182405.6,HOG:D0622680.2a.5b,piggyBac transposable element derived 4 [Sour...,None
85282,HUMAN071311,ENSP00000265260.3,ENST00000265260.8,ENSG00000081154.12,HOG:D0629401.1c.1d.2d.2b.1b,PEST proteolytic signal containing nuclear pr...,None


In [41]:
oma_human_to_zfish['ENSG']=[re.sub("\..+", "", x) for x in oma_human_to_zfish.ens_hg.str.split("|",expand=True)[3]]
oma_human_to_zfish['ENSG']=[re.sub(" ", "", x) for x in oma_human_to_zfish['ENSG']]

oma_human_to_zfish['GeneID_hg']=oma_human_to_zfish.ens_hg.str.split("|",expand=True)[3]
oma_human_to_zfish['GeneProduct_hg']=oma_human_to_zfish.ens_hg.str.split("|",expand=True)[3]

oma_human_to_zfish

,chr,position,ens_zf,ens_hg,n_orth,score,ENSG,GeneID_hg,GeneProduct_hg
0,1,28095,DANRE000001 | ENSDARP00000124749.1 | ENSDART00...,HUMAN028095 | ENSP00000378288.4 | ENST00000394...,1:1,13142.0,ENSG00000140795,ENSG00000140795.13,ENSG00000140795.13
1,2,96889,DANRE000002 | ENSDARP00000118783.1 | ENSDART00...,HUMAN096889 | ENSP00000367880.3 | ENST00000378...,1:1,4537.0,ENSG00000165282,ENSG00000165282.14,ENSG00000165282.14
2,4,59690,DANRE000004 | ENSDARP00000147774.1 | ENSDART00...,HUMAN059690 | ENSP00000248933.6 | ENST00000248...,1:1,2061.0,ENSG00000100095,ENSG00000100095.19,ENSG00000100095.19
3,8,30660,DANRE000008 | ENSDARP00000156564.1 | ENSDART00...,HUMAN030660 | ENSP00000301335.5 | ENST00000301...,many:1,NaN,ENSG00000167703,ENSG00000167703.15,ENSG00000167703.15
4,12,35462,DANRE000012 | ENSDARP00000152677.1 | ENSDART00...,HUMAN035462 | ENSP00000380888.2 | ENST00000397...,many:1,NaN,ENSG00000108510,ENSG00000108510.10,ENSG00000108510.10
...,...,...,...,...,...,...,...,...,...
85279,52067,11378,DANRE052067 | ENSDARP00000141076.2 | ENSDART00...,HUMAN011378 | ENSP00000261167.2 | ENST00000261...,many:1,NaN,ENSG00000084463,ENSG00000084463.8,ENSG00000084463.8
85280,52070,80350,DANRE052070 | ENSDARP00000137807.2 | ENSDART00...,HUMAN080350 | ENSP00000250535.4 | ENST00000250...,1:1,9222.0,ENSG00000129596,ENSG00000129596.5,ENSG00000129596.5
85281,52075,22103,DANRE052075 | ENSDARP00000152484.1 | ENSDART00...,HUMAN022103 | ENSP00000380872.2 | ENST00000397...,many:1,11846.0,ENSG00000182405,ENSG00000182405.6,ENSG00000182405.6
85282,52083,71311,DANRE052083 | ENSDARP00000149472.1 | ENSDART00...,HUMAN071311 | ENSP00000265260.3 | ENST00000265...,many:1,11738.0,ENSG00000081154,ENSG00000081154.12,ENSG00000081154.12


In [42]:
oma_human_to_zfish.ens_zf.str.split("|",expand=True)


,0,1,2,3,4,5,6
0,DANRE000001,ENSDARP00000124749.1,ENSDART00000148606.2,ENSDARG00000078271.4,HOG:D0571967.1a,si:ch211-57m13.8 [Source:ZFIN;Acc:ZDB-GENE-11...,None
1,DANRE000002,ENSDARP00000118783.1,ENSDART00000135044.2,ENSDARG00000011743.9,E9QI13,HOG:D0659026.1b,phosphatidylinositol glycan anchor biosynthes...
2,DANRE000004,ENSDARP00000147774.1,ENSDART00000190331.1,ENSDARG00000100594.2,A0A2R8Q401,HOG:D0629707.2a,seizure related 6 homolog (mouse)-like [Sourc...
3,DANRE000008,ENSDARP00000156564.1,ENSDART00000180013.1,ENSDARG00000061120.8,A0A2R8QSV3,HOG:D0684981.2a.3b.4a.11a.10b,solute carrier family 43 member 2b [Source:ZF...
4,DANRE000012,ENSDARP00000152677.1,ENSDART00000190836.1,ENSDARG00000053884.8,HOG:D0659262.2b.6b.2a.1b.2b,mediator complex subunit 13a [Source:ZFIN;Acc...,None
...,...,...,...,...,...,...,...
85279,DANRE052067,ENSDARP00000141076.2,ENSDART00000157723.2,ENSDARG00000099151.2,HOG:D0673655.1c.6a.1a.3a,WW domain binding protein 11 [Source:NCBI gen...,None
85280,DANRE052070,ENSDARP00000137807.2,ENSDART00000160352.2,ENSDARG00000099389.2,A0A0R4IS75,HOG:D0690063.3b.9a,"cysteine dioxygenase, type I [Source:NCBI gen..."
85281,DANRE052075,ENSDARP00000152484.1,ENSDART00000191202.1,ENSDARG00000115161.1,HOG:D0584301.1c,transcript_id=ENSDART00000191202.1,None
85282,DANRE052083,ENSDARP00000149472.1,ENSDART00000187524.1,ENSDARG00000116774.1,HOG:D0629401.3b,PEST proteolytic signal-containing nuclear pr...,None


In [43]:

oma_human_to_zfish['ENSD']=[re.sub("\..+", "", x) for x in oma_human_to_zfish.ens_zf.str.split("|",expand=True)[3]]
oma_human_to_zfish['ENSD']=[re.sub(" ", "", x) for x in oma_human_to_zfish['ENSD']]

oma_human_to_zfish['GeneID_zf']=oma_human_to_zfish.ens_hg.str.split("|",expand=True)[3]
oma_human_to_zfish['GeneProduct_zf']=oma_human_to_zfish.ens_hg.str.split("|",expand=True)[3]

oma_human_to_zfish

,chr,position,ens_zf,ens_hg,n_orth,score,ENSG,GeneID_hg,GeneProduct_hg,ENSD,GeneID_zf,GeneProduct_zf
0,1,28095,DANRE000001 | ENSDARP00000124749.1 | ENSDART00...,HUMAN028095 | ENSP00000378288.4 | ENST00000394...,1:1,13142.0,ENSG00000140795,ENSG00000140795.13,ENSG00000140795.13,ENSDARG00000078271,ENSG00000140795.13,ENSG00000140795.13
1,2,96889,DANRE000002 | ENSDARP00000118783.1 | ENSDART00...,HUMAN096889 | ENSP00000367880.3 | ENST00000378...,1:1,4537.0,ENSG00000165282,ENSG00000165282.14,ENSG00000165282.14,ENSDARG00000011743,ENSG00000165282.14,ENSG00000165282.14
2,4,59690,DANRE000004 | ENSDARP00000147774.1 | ENSDART00...,HUMAN059690 | ENSP00000248933.6 | ENST00000248...,1:1,2061.0,ENSG00000100095,ENSG00000100095.19,ENSG00000100095.19,ENSDARG00000100594,ENSG00000100095.19,ENSG00000100095.19
3,8,30660,DANRE000008 | ENSDARP00000156564.1 | ENSDART00...,HUMAN030660 | ENSP00000301335.5 | ENST00000301...,many:1,NaN,ENSG00000167703,ENSG00000167703.15,ENSG00000167703.15,ENSDARG00000061120,ENSG00000167703.15,ENSG00000167703.15
4,12,35462,DANRE000012 | ENSDARP00000152677.1 | ENSDART00...,HUMAN035462 | ENSP00000380888.2 | ENST00000397...,many:1,NaN,ENSG00000108510,ENSG00000108510.10,ENSG00000108510.10,ENSDARG00000053884,ENSG00000108510.10,ENSG00000108510.10
...,...,...,...,...,...,...,...,...,...,...,...,...
85279,52067,11378,DANRE052067 | ENSDARP00000141076.2 | ENSDART00...,HUMAN011378 | ENSP00000261167.2 | ENST00000261...,many:1,NaN,ENSG00000084463,ENSG00000084463.8,ENSG00000084463.8,ENSDARG00000099151,ENSG00000084463.8,ENSG00000084463.8
85280,52070,80350,DANRE052070 | ENSDARP00000137807.2 | ENSDART00...,HUMAN080350 | ENSP00000250535.4 | ENST00000250...,1:1,9222.0,ENSG00000129596,ENSG00000129596.5,ENSG00000129596.5,ENSDARG00000099389,ENSG00000129596.5,ENSG00000129596.5
85281,52075,22103,DANRE052075 | ENSDARP00000152484.1 | ENSDART00...,HUMAN022103 | ENSP00000380872.2 | ENST00000397...,many:1,11846.0,ENSG00000182405,ENSG00000182405.6,ENSG00000182405.6,ENSDARG00000115161,ENSG00000182405.6,ENSG00000182405.6
85282,52083,71311,DANRE052083 | ENSDARP00000149472.1 | ENSDART00...,HUMAN071311 | ENSP00000265260.3 | ENST00000265...,many:1,11738.0,ENSG00000081154,ENSG00000081154.12,ENSG00000081154.12,ENSDARG00000116774,ENSG00000081154.12,ENSG00000081154.12


In [44]:

oma_human_to_zfish=oma_human_to_zfish.merge(ensembl_human,left_on="ENSG",right_on='Gene stable ID')
oma_human_to_zfish





,chr,position,ens_zf,ens_hg,n_orth,score,ENSG,GeneID_hg,GeneProduct_hg,ENSD,GeneID_zf,GeneProduct_zf,Gene stable ID,Gene stable ID version,HGNC ID,HGNC symbol
0,1,28095,DANRE000001 | ENSDARP00000124749.1 | ENSDART00...,HUMAN028095 | ENSP00000378288.4 | ENST00000394...,1:1,13142.0,ENSG00000140795,ENSG00000140795.13,ENSG00000140795.13,ENSDARG00000078271,ENSG00000140795.13,ENSG00000140795.13,ENSG00000140795,ENSG00000140795.13,HGNC:29826,MYLK3
1,2,96889,DANRE000002 | ENSDARP00000118783.1 | ENSDART00...,HUMAN096889 | ENSP00000367880.3 | ENST00000378...,1:1,4537.0,ENSG00000165282,ENSG00000165282.14,ENSG00000165282.14,ENSDARG00000011743,ENSG00000165282.14,ENSG00000165282.14,ENSG00000165282,ENSG00000165282.15,HGNC:23215,PIGO
2,4,59690,DANRE000004 | ENSDARP00000147774.1 | ENSDART00...,HUMAN059690 | ENSP00000248933.6 | ENST00000248...,1:1,2061.0,ENSG00000100095,ENSG00000100095.19,ENSG00000100095.19,ENSDARG00000100594,ENSG00000100095.19,ENSG00000100095.19,ENSG00000100095,ENSG00000100095.19,HGNC:10763,SEZ6L
3,8,30660,DANRE000008 | ENSDARP00000156564.1 | ENSDART00...,HUMAN030660 | ENSP00000301335.5 | ENST00000301...,many:1,NaN,ENSG00000167703,ENSG00000167703.15,ENSG00000167703.15,ENSDARG00000061120,ENSG00000167703.15,ENSG00000167703.15,ENSG00000167703,ENSG00000167703.15,HGNC:23087,SLC43A2
4,8642,30660,DANRE008642 | ENSDARP00000138218.1 | ENSDART00...,HUMAN030660 | ENSP00000301335.5 | ENST00000301...,many:1,2729.0,ENSG00000167703,ENSG00000167703.15,ENSG00000167703.15,ENSDARG00000036848,ENSG00000167703.15,ENSG00000167703.15,ENSG00000167703,ENSG00000167703.15,HGNC:23087,SLC43A2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84955,52011,23221,DANRE052011 | ENSDARP00000130348.2 | ENSDART00...,HUMAN023221 | ENSP00000403400.3 | ENST00000442...,1:1,8512.0,ENSG00000260916,ENSG00000260916.7,ENSG00000260916.7,ENSDARG00000102110,ENSG00000260916.7,ENSG00000260916.7,ENSG00000260916,ENSG00000260916.8,HGNC:24227,CCPG1
84956,52045,84143,DANRE052045 | ENSDARP00000140934.2 | ENSDART00...,HUMAN084143 | ENSP00000364349.3 | ENST00000375...,1:1,10034.0,ENSG00000213676,ENSG00000213676.13,ENSG00000213676.13,ENSDARG00000101369,ENSG00000213676.13,ENSG00000213676.13,ENSG00000213676,ENSG00000213676.13,HGNC:2349,ATF6B
84957,52063,81925,DANRE052063 | ENSDARP00000132758.1 | ENSDART00...,HUMAN081925 | ENSP00000297107.6 | ENST00000297...,1:1,4127.0,ENSG00000164574,ENSG00000164574.16,ENSG00000164574.16,ENSDARG00000100453,ENSG00000164574.16,ENSG00000164574.16,ENSG00000164574,ENSG00000164574.16,HGNC:19873,GALNT10
84958,52070,80350,DANRE052070 | ENSDARP00000137807.2 | ENSDART00...,HUMAN080350 | ENSP00000250535.4 | ENST00000250...,1:1,9222.0,ENSG00000129596,ENSG00000129596.5,ENSG00000129596.5,ENSDARG00000099389,ENSG00000129596.5,ENSG00000129596.5,ENSG00000129596,ENSG00000129596.5,HGNC:1795,CDO1


In [45]:
ensembl_human

,Gene stable ID,Gene stable ID version,HGNC ID,HGNC symbol
0,ENSG00000210049,ENSG00000210049.1,HGNC:7481,MT-TF
1,ENSG00000211459,ENSG00000211459.2,HGNC:7470,MT-RNR1
2,ENSG00000210077,ENSG00000210077.1,HGNC:7500,MT-TV
3,ENSG00000210082,ENSG00000210082.2,HGNC:7471,MT-RNR2
4,ENSG00000209082,ENSG00000209082.1,HGNC:7490,MT-TL1
...,...,...,...,...
70121,ENSG00000236500,ENSG00000236500.1,HGNC:1646,CD24P1
70122,ENSG00000197312,ENSG00000197312.13,HGNC:24578,DDI2
70123,ENSG00000215695,ENSG00000215695.2,HGNC:10458,RSC1A1
70124,ENSG00000271742,ENSG00000271742.1,NaN,NaN


In [46]:
oma_human_to_zfish.ENSD[1]


'ENSDARG00000011743'

In [47]:
ensembl_zfish['Zebrafish gene stable ID'][1]

'ENSDARG00000104512'

In [48]:

oma_human_to_zfish=oma_human_to_zfish.merge(ensembl_zfish,left_on="ENSD",right_on='Zebrafish gene stable ID')

oma_human_to_zfish

,chr,position,ens_zf,ens_hg,n_orth,score,ENSG,GeneID_hg,GeneProduct_hg,ENSD,GeneID_zf,GeneProduct_zf,Gene stable ID,Gene stable ID version_x,HGNC ID,HGNC symbol,Zebrafish gene stable ID,Gene stable ID version_y,ZFIN ID,ZFIN symbol
0,1,28095,DANRE000001 | ENSDARP00000124749.1 | ENSDART00...,HUMAN028095 | ENSP00000378288.4 | ENST00000394...,1:1,13142.0,ENSG00000140795,ENSG00000140795.13,ENSG00000140795.13,ENSDARG00000078271,ENSG00000140795.13,ENSG00000140795.13,ENSG00000140795,ENSG00000140795.13,HGNC:29826,MYLK3,ENSDARG00000078271,ENSDARG00000078271.4,NaN,NaN
1,2,96889,DANRE000002 | ENSDARP00000118783.1 | ENSDART00...,HUMAN096889 | ENSP00000367880.3 | ENST00000378...,1:1,4537.0,ENSG00000165282,ENSG00000165282.14,ENSG00000165282.14,ENSDARG00000011743,ENSG00000165282.14,ENSG00000165282.14,ENSG00000165282,ENSG00000165282.15,HGNC:23215,PIGO,ENSDARG00000011743,ENSDARG00000011743.9,NaN,NaN
2,4,59690,DANRE000004 | ENSDARP00000147774.1 | ENSDART00...,HUMAN059690 | ENSP00000248933.6 | ENST00000248...,1:1,2061.0,ENSG00000100095,ENSG00000100095.19,ENSG00000100095.19,ENSDARG00000100594,ENSG00000100095.19,ENSG00000100095.19,ENSG00000100095,ENSG00000100095.19,HGNC:10763,SEZ6L,ENSDARG00000100594,ENSDARG00000100594.2,NaN,NaN
3,8,30660,DANRE000008 | ENSDARP00000156564.1 | ENSDART00...,HUMAN030660 | ENSP00000301335.5 | ENST00000301...,many:1,NaN,ENSG00000167703,ENSG00000167703.15,ENSG00000167703.15,ENSDARG00000061120,ENSG00000167703.15,ENSG00000167703.15,ENSG00000167703,ENSG00000167703.15,HGNC:23087,SLC43A2,ENSDARG00000061120,ENSDARG00000061120.8,ZDB-GENE-041212-6,slc43a2b
4,8642,30660,DANRE008642 | ENSDARP00000138218.1 | ENSDART00...,HUMAN030660 | ENSP00000301335.5 | ENST00000301...,many:1,2729.0,ENSG00000167703,ENSG00000167703.15,ENSG00000167703.15,ENSDARG00000036848,ENSG00000167703.15,ENSG00000167703.15,ENSG00000167703,ENSG00000167703.15,HGNC:23087,SLC43A2,ENSDARG00000036848,ENSDARG00000036848.9,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85219,52011,23221,DANRE052011 | ENSDARP00000130348.2 | ENSDART00...,HUMAN023221 | ENSP00000403400.3 | ENST00000442...,1:1,8512.0,ENSG00000260916,ENSG00000260916.7,ENSG00000260916.7,ENSDARG00000102110,ENSG00000260916.7,ENSG00000260916.7,ENSG00000260916,ENSG00000260916.8,HGNC:24227,CCPG1,ENSDARG00000102110,ENSDARG00000102110.2,NaN,NaN
85220,52045,84143,DANRE052045 | ENSDARP00000140934.2 | ENSDART00...,HUMAN084143 | ENSP00000364349.3 | ENST00000375...,1:1,10034.0,ENSG00000213676,ENSG00000213676.13,ENSG00000213676.13,ENSDARG00000101369,ENSG00000213676.13,ENSG00000213676.13,ENSG00000213676,ENSG00000213676.13,HGNC:2349,ATF6B,ENSDARG00000101369,ENSDARG00000101369.2,NaN,NaN
85221,52063,81925,DANRE052063 | ENSDARP00000132758.1 | ENSDART00...,HUMAN081925 | ENSP00000297107.6 | ENST00000297...,1:1,4127.0,ENSG00000164574,ENSG00000164574.16,ENSG00000164574.16,ENSDARG00000100453,ENSG00000164574.16,ENSG00000164574.16,ENSG00000164574,ENSG00000164574.16,HGNC:19873,GALNT10,ENSDARG00000100453,ENSDARG00000100453.2,NaN,NaN
85222,52070,80350,DANRE052070 | ENSDARP00000137807.2 | ENSDART00...,HUMAN080350 | ENSP00000250535.4 | ENST00000250...,1:1,9222.0,ENSG00000129596,ENSG00000129596.5,ENSG00000129596.5,ENSDARG00000099389,ENSG00000129596.5,ENSG00000129596.5,ENSG00000129596,ENSG00000129596.5,HGNC:1795,CDO1,ENSDARG00000099389,ENSDARG00000099389.2,ZDB-GENE-040426-1704,cdo1


In [49]:

oma_human_to_zfish=oma_human_to_zfish.dropna(subset=['HGNC symbol', 'ZFIN symbol'])
oma_human_to_zfish=oma_human_to_zfish.rename(columns={'HGNC symbol':'hg_symbol','ZFIN symbol':'zf_symbol'})



In [50]:
oma_human_to_zfish.columns

Index(['chr', 'position', 'ens_zf', 'ens_hg', 'n_orth', 'score', 'ENSG',
       'GeneID_hg', 'GeneProduct_hg', 'ENSD', 'GeneID_zf', 'GeneProduct_zf',
       'Gene stable ID', 'Gene stable ID version_x', 'HGNC ID', 'hg_symbol',
       'Zebrafish gene stable ID', 'Gene stable ID version_y', 'ZFIN ID',
       'zf_symbol'],
      dtype='object')

In [51]:
oma_human_to_zfish[oma_human_to_zfish.hg_symbol.str.match(r"MET")]

,chr,position,ens_zf,ens_hg,n_orth,score,ENSG,GeneID_hg,GeneProduct_hg,ENSD,GeneID_zf,GeneProduct_zf,Gene stable ID,Gene stable ID version_x,HGNC ID,hg_symbol,Zebrafish gene stable ID,Gene stable ID version_y,ZFIN ID,zf_symbol
54787,2322,13868,DANRE002322 | ENSDARP00000128994.2 | ENSDART00...,HUMAN013868 | ENSP00000314441.7 | ENST00000324...,1:1,8064.0,ENSG00000037897,ENSG00000037897.17,ENSG00000037897.17,ENSDARG00000076518,ENSG00000037897.17,ENSG00000037897.17,ENSG00000037897,ENSG00000037897.17,HGNC:7030,METTL1,ENSDARG00000076518,ENSDARG00000076518.6,ZDB-GENE-041010-24,mettl1
56633,4420,27147,DANRE004420 | ENSDARP00000107001.2 | ENSDART00...,HUMAN027147 | ENSP00000350874.3 | ENST00000358...,1:1,8628.0,ENSG00000197006,ENSG00000197006.14,ENSG00000197006.14,ENSDARG00000060001,ENSG00000197006.14,ENSG00000197006.14,ENSG00000197006,ENSG00000197006.15,HGNC:24586,METTL9,ENSDARG00000060001,ENSDARG00000060001.7,ZDB-GENE-061013-752,mettl9
57506,5826,76665,DANRE005826 | ENSDARP00000113818.1 | ENSDART00...,HUMAN076665 | ENSP00000296411.6 | ENST00000296...,1:1,3620.0,ENSG00000164024,ENSG00000164024.12,ENSG00000164024.12,ENSDARG00000033440,ENSG00000164024.12,ENSG00000164024.12,ENSG00000164024,ENSG00000164024.12,HGNC:15789,METAP1,ENSDARG00000033440,ENSDARG00000033440.10,ZDB-GENE-050626-124,metap1
60289,8775,30766,DANRE008775 | ENSDARP00000072692.4 | ENSDART00...,HUMAN030766 | ENSP00000263092.5 | ENST00000263...,1:1,4091.0,ENSG00000127804,ENSG00000127804.13,ENSG00000127804.13,ENSDARG00000055838,ENSG00000127804.13,ENSG00000127804.13,ENSG00000127804,ENSG00000127804.13,HGNC:28484,METTL16,ENSDARG00000055838,ENSDARG00000055838.6,ZDB-GENE-040801-130,mettl16
68215,10681,68732,DANRE010681 | ENSDARP00000093347.3 | ENSDART00...,HUMAN068732 | ENSP00000373300.3 | ENST00000383...,1:1,7710.0,ENSG00000206562,ENSG00000206562.12,ENSG00000206562.12,ENSDARG00000070085,ENSG00000206562.12,ENSG00000206562.12,ENSG00000206562,ENSG00000206562.12,HGNC:28343,METTL6,ENSDARG00000070085,ENSDARG00000070085.4,ZDB-GENE-040426-926,mettl6
68549,11277,52804,DANRE011277 | ENSDARP00000134885.1 | ENSDART00...,HUMAN052804 | ENSP00000357199.4 | ENST00000368...,1:1,8200.0,ENSG00000143303,ENSG00000143303.12,ENSG00000143303.12,ENSDARG00000089033,ENSG00000143303.12,ENSG00000143303.12,ENSG00000143303,ENSG00000143303.12,HGNC:24273,METTL25B,ENSDARG00000089033,ENSDARG00000089033.5,ZDB-GENE-050522-515,mettl25b
71717,44445,17565,DANRE044445 | ENSDARP00000107813.2 | ENSDART00...,HUMAN017565 | ENSP00000267273.5 | ENST00000267...,many:1,NaN,ENSG00000139780,ENSG00000139780.7,ENSG00000139780.7,ENSDARG00000055437,ENSG00000139780.7,ENSG00000139780.7,ENSG00000139780,ENSG00000139780.8,HGNC:33717,METTL21C,ENSDARG00000055437,ENSDARG00000055437.8,ZDB-GENE-080204-102,mettl21cb
71718,45977,17565,DANRE045977 | ENSDARP00000147170.1 | ENSDART00...,HUMAN017565 | ENSP00000267273.5 | ENST00000267...,many:1,11219.0,ENSG00000139780,ENSG00000139780.7,ENSG00000139780.7,ENSDARG00000112856,ENSG00000139780.7,ENSG00000139780.7,ENSG00000139780,ENSG00000139780.8,HGNC:33717,METTL21C,ENSDARG00000112856,ENSDARG00000112856.1,ZDB-GENE-080204-102,mettl21cb
71946,17746,65562,DANRE017746 | ENSDARP00000010471.6 | ENSDART00...,HUMAN065562 | ENSP00000315152.4 | ENST00000315...,many:1,NaN,ENSG00000172878,ENSG00000172878.14,ENSG00000172878.14,ENSDARG00000019715,ENSG00000172878.14,ENSG00000172878.14,ENSG00000172878,ENSG00000172878.14,HGNC:32583,METAP1D,ENSDARG00000019715,ENSDARG00000019715.9,ZDB-GENE-050522-71,metap1d
71947,46634,65562,DANRE046634 | ENSDARP00000147735.1 | ENSDART00...,HUMAN065562 | ENSP00000315152.4 | ENST00000315...,many:1,7069.0,ENSG00000172878,ENSG00000172878.14,ENSG00000172878.14,ENSDARG00000112086,ENSG00000172878.14,ENSG00000172878.14,ENSG00000172878,ENSG00000172878.14,HGNC:32583,METAP1D,ENSDARG00000112086,ENSDARG00000112086.1,ZDB-GENE-050522-71,metap1d


In [52]:

#prepare alliance db
alliance_human_to_zfish=alliance_human_to_zfish[alliance_human_to_zfish["Gene1SpeciesName"]=="Homo sapiens"] 
alliance_human_to_zfish=alliance_human_to_zfish[alliance_human_to_zfish["Gene2SpeciesName"]=="Danio rerio"]
alliance_human_to_zfish=alliance_human_to_zfish.rename(columns={'Gene1Symbol':'hg_symbol','Gene2Symbol':'zf_symbol'})


In [53]:
alliance_human_to_zfish[alliance_human_to_zfish.hg_symbol.str.contains(r'MET')]

,Gene1ID,hg_symbol,Gene1SpeciesTaxonID,Gene1SpeciesName,Gene2ID,zf_symbol,Gene2SpeciesTaxonID,Gene2SpeciesName,Algorithms,AlgorithmsMatch,OutOfAlgorithms,IsBestScore,IsBestRevScore
292943,HGNC:14141,METTL26,NCBITaxon:9606,Homo sapiens,ZFIN:ZDB-GENE-031118-200,mettl26,NCBITaxon:7955,Danio rerio,OrthoInspector|OMA|OrthoFinder|SonicParanoid|H...,8,10,Yes,Yes
298816,HGNC:7029,MET,NCBITaxon:9606,Homo sapiens,ZFIN:ZDB-GENE-041014-1,met,NCBITaxon:7955,Danio rerio,OrthoInspector|OMA|OrthoFinder|SonicParanoid|H...,10,10,Yes,Yes
307567,HGNC:26228,METTL25,NCBITaxon:9606,Homo sapiens,ZFIN:ZDB-GENE-041210-90,mettl25,NCBITaxon:7955,Danio rerio,OrthoInspector|OMA|OrthoFinder|SonicParanoid|H...,10,10,Yes,Yes
310504,HGNC:16672,METAP2,NCBITaxon:9606,Homo sapiens,ZFIN:ZDB-GENE-061013-5,metap2a,NCBITaxon:7955,Danio rerio,OMA|OrthoFinder|ZFIN|PhylomeDB|PANTHER,5,10,No,Yes
310976,HGNC:19068,METTL27,NCBITaxon:9606,Homo sapiens,ZFIN:ZDB-GENE-060421-5918,mettl27,NCBITaxon:7955,Danio rerio,OrthoInspector|OMA|OrthoFinder|SonicParanoid|H...,10,10,Yes,Yes
345311,HGNC:17563,METTL3,NCBITaxon:9606,Homo sapiens,ZFIN:ZDB-GENE-030131-9498,mettl3,NCBITaxon:7955,Danio rerio,OrthoInspector|OMA|OrthoFinder|Hieranoid|Sonic...,9,10,Yes,Yes
351602,HGNC:28343,METTL6,NCBITaxon:9606,Homo sapiens,ZFIN:ZDB-GENE-040426-926,mettl6,NCBITaxon:7955,Danio rerio,OrthoInspector|OMA|OrthoFinder|Hieranoid|Sonic...,10,10,Yes,Yes
353490,HGNC:41948,METTL21EP,NCBITaxon:9606,Homo sapiens,ZFIN:ZDB-GENE-040718-53,mettl21e,NCBITaxon:7955,Danio rerio,OrthoInspector|OMA|OrthoFinder|Hieranoid|Sonic...,9,10,Yes,Yes
353782,HGNC:28484,METTL16,NCBITaxon:9606,Homo sapiens,ZFIN:ZDB-GENE-040801-130,mettl16,NCBITaxon:7955,Danio rerio,OrthoInspector|OMA|OrthoFinder|Hieranoid|Sonic...,10,10,Yes,Yes
356261,HGNC:25856,METTL8,NCBITaxon:9606,Homo sapiens,ZFIN:ZDB-GENE-041114-19,mettl8,NCBITaxon:7955,Danio rerio,OMA|OrthoFinder|Hieranoid|ZFIN|PhylomeDB|PANTH...,7,10,Yes,Yes


In [54]:

#combine orthologies
gene_orthologies=pd.concat([ens_human_zf_mapping.iloc[:,[2,4]],alliance_human_to_zfish.iloc[:,[1,5]],oma_human_to_zfish.iloc[:,[15,19]]])
#gene_orthologies=pd.concat([ens_human_zf_mapping.iloc[:,[2,4]],alliance_human_to_zfish.iloc[:,[1,5]]])
gene_orthologies





,hg_symbol,zf_symbol
13,AACS,aacs
24,AAMDC,aamdc
25,AAMP,aamp
27,AAR2,aar2
32,AARSD1,aarsd1
...,...,...
85209,TRIM32,trim32
85210,STYXL2,dusp27
85213,CREG1,creg1
85218,LEPROTL1,zgc:92045


In [55]:


gene_orthologies[gene_orthologies.zf_symbol.str.match(r"met",na=False)]

,hg_symbol,zf_symbol
17542,METAP2,metap2b
17546,METTL1,mettl1
17547,METTL13,mettl13
17550,METTL15,mettl15
17553,METTL16,mettl16
...,...,...
82618,METTL7B,mettl7a.1
83333,METTL15,mettl15
83349,METTL3,mettl3
84601,METTL5,mettl5


In [56]:
aerts_tbl=aerts_human_tbl.merge(gene_orthologies,left_on=aerts_human_tbl["gene_name"].str.lower(),right_on=gene_orthologies['hg_symbol'].str.lower())
aerts_tbl

,key_0,#motif_id,motif_name,motif_description,source_name,source_version,gene_name,motif_similarity_qvalue,similar_motif_id,similar_motif_description,orthologous_identity,orthologous_gene_name,orthologous_species,description,hg_symbol,zf_symbol
0,hnf4a,metacluster_196.3,EcR_usp,EcR/usp,bergman,1.1,HNF4A,0.0,None,None,0.270042,FBgn0003964,D. melanogaster,gene is orthologous to FBgn0003964 in D. melan...,HNF4A,hnf4a
1,hnf4a,metacluster_196.3,EcR_usp,EcR/usp,bergman,1.1,HNF4A,0.0,None,None,0.270042,FBgn0003964,D. melanogaster,gene is orthologous to FBgn0003964 in D. melan...,HNF4A,hnf4a
2,hnf4a,metacluster_121.1,usp,usp,bergman,1.1,HNF4A,0.0,None,None,0.270042,FBgn0003964,D. melanogaster,gene is orthologous to FBgn0003964 in D. melan...,HNF4A,hnf4a
3,hnf4a,metacluster_121.1,usp,usp,bergman,1.1,HNF4A,0.0,None,None,0.270042,FBgn0003964,D. melanogaster,gene is orthologous to FBgn0003964 in D. melan...,HNF4A,hnf4a
4,hnf4a,cisbp__M00182,M00182,Hnf4a[gene ID: ENSMUSG00000017950 species: Mus...,cisbp,2.00,HNF4A,0.0,None,None,0.957806,ENSMUSG00000017950,M. musculus,gene is orthologous to ENSMUSG00000017950 in M...,HNF4A,hnf4a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633900,yod1,yetfasco__YFL044C_1166,YFL044C_1166,YFL044C_1166,yetfasco,1.02,YOD1,0.0,None,None,0.312292,YFL044C,S. cerevisiae,motif is annotated for orthologous gene YFL044...,YOD1,yod1
633901,yod1,yetfasco__YFL044C_1166,YFL044C_1166,YFL044C_1166,yetfasco,1.02,YOD1,0.0,None,None,0.312292,YFL044C,S. cerevisiae,motif is annotated for orthologous gene YFL044...,YOD1,yod1
633902,mettl3,yetfasco__YGL192W_1000,YGL192W_1000,YGL192W_1000,yetfasco,1.02,METTL3,0.0,None,None,0.320690,YGL192W,S. cerevisiae,gene is orthologous to YGL192W in S. cerevisia...,METTL3,mettl3
633903,mettl3,yetfasco__YGL192W_1000,YGL192W_1000,YGL192W_1000,yetfasco,1.02,METTL3,0.0,None,None,0.320690,YGL192W,S. cerevisiae,gene is orthologous to YGL192W in S. cerevisia...,METTL3,mettl3


In [57]:
aerts_tbl

,key_0,#motif_id,motif_name,motif_description,source_name,source_version,gene_name,motif_similarity_qvalue,similar_motif_id,similar_motif_description,orthologous_identity,orthologous_gene_name,orthologous_species,description,hg_symbol,zf_symbol
0,hnf4a,metacluster_196.3,EcR_usp,EcR/usp,bergman,1.1,HNF4A,0.0,None,None,0.270042,FBgn0003964,D. melanogaster,gene is orthologous to FBgn0003964 in D. melan...,HNF4A,hnf4a
1,hnf4a,metacluster_196.3,EcR_usp,EcR/usp,bergman,1.1,HNF4A,0.0,None,None,0.270042,FBgn0003964,D. melanogaster,gene is orthologous to FBgn0003964 in D. melan...,HNF4A,hnf4a
2,hnf4a,metacluster_121.1,usp,usp,bergman,1.1,HNF4A,0.0,None,None,0.270042,FBgn0003964,D. melanogaster,gene is orthologous to FBgn0003964 in D. melan...,HNF4A,hnf4a
3,hnf4a,metacluster_121.1,usp,usp,bergman,1.1,HNF4A,0.0,None,None,0.270042,FBgn0003964,D. melanogaster,gene is orthologous to FBgn0003964 in D. melan...,HNF4A,hnf4a
4,hnf4a,cisbp__M00182,M00182,Hnf4a[gene ID: ENSMUSG00000017950 species: Mus...,cisbp,2.00,HNF4A,0.0,None,None,0.957806,ENSMUSG00000017950,M. musculus,gene is orthologous to ENSMUSG00000017950 in M...,HNF4A,hnf4a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633900,yod1,yetfasco__YFL044C_1166,YFL044C_1166,YFL044C_1166,yetfasco,1.02,YOD1,0.0,None,None,0.312292,YFL044C,S. cerevisiae,motif is annotated for orthologous gene YFL044...,YOD1,yod1
633901,yod1,yetfasco__YFL044C_1166,YFL044C_1166,YFL044C_1166,yetfasco,1.02,YOD1,0.0,None,None,0.312292,YFL044C,S. cerevisiae,motif is annotated for orthologous gene YFL044...,YOD1,yod1
633902,mettl3,yetfasco__YGL192W_1000,YGL192W_1000,YGL192W_1000,yetfasco,1.02,METTL3,0.0,None,None,0.320690,YGL192W,S. cerevisiae,gene is orthologous to YGL192W in S. cerevisia...,METTL3,mettl3
633903,mettl3,yetfasco__YGL192W_1000,YGL192W_1000,YGL192W_1000,yetfasco,1.02,METTL3,0.0,None,None,0.320690,YGL192W,S. cerevisiae,gene is orthologous to YGL192W in S. cerevisia...,METTL3,mettl3


In [58]:
aerts_tbl['gene_name']=aerts_tbl['zf_symbol']
aerts_tbl=aerts_tbl.drop(columns=['hg_symbol', 'zf_symbol'])

aerts_tbl

,key_0,#motif_id,motif_name,motif_description,source_name,source_version,gene_name,motif_similarity_qvalue,similar_motif_id,similar_motif_description,orthologous_identity,orthologous_gene_name,orthologous_species,description
0,hnf4a,metacluster_196.3,EcR_usp,EcR/usp,bergman,1.1,hnf4a,0.0,None,None,0.270042,FBgn0003964,D. melanogaster,gene is orthologous to FBgn0003964 in D. melan...
1,hnf4a,metacluster_196.3,EcR_usp,EcR/usp,bergman,1.1,hnf4a,0.0,None,None,0.270042,FBgn0003964,D. melanogaster,gene is orthologous to FBgn0003964 in D. melan...
2,hnf4a,metacluster_121.1,usp,usp,bergman,1.1,hnf4a,0.0,None,None,0.270042,FBgn0003964,D. melanogaster,gene is orthologous to FBgn0003964 in D. melan...
3,hnf4a,metacluster_121.1,usp,usp,bergman,1.1,hnf4a,0.0,None,None,0.270042,FBgn0003964,D. melanogaster,gene is orthologous to FBgn0003964 in D. melan...
4,hnf4a,cisbp__M00182,M00182,Hnf4a[gene ID: ENSMUSG00000017950 species: Mus...,cisbp,2.00,hnf4a,0.0,None,None,0.957806,ENSMUSG00000017950,M. musculus,gene is orthologous to ENSMUSG00000017950 in M...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633900,yod1,yetfasco__YFL044C_1166,YFL044C_1166,YFL044C_1166,yetfasco,1.02,yod1,0.0,None,None,0.312292,YFL044C,S. cerevisiae,motif is annotated for orthologous gene YFL044...
633901,yod1,yetfasco__YFL044C_1166,YFL044C_1166,YFL044C_1166,yetfasco,1.02,yod1,0.0,None,None,0.312292,YFL044C,S. cerevisiae,motif is annotated for orthologous gene YFL044...
633902,mettl3,yetfasco__YGL192W_1000,YGL192W_1000,YGL192W_1000,yetfasco,1.02,mettl3,0.0,None,None,0.320690,YGL192W,S. cerevisiae,gene is orthologous to YGL192W in S. cerevisia...
633903,mettl3,yetfasco__YGL192W_1000,YGL192W_1000,YGL192W_1000,yetfasco,1.02,mettl3,0.0,None,None,0.320690,YGL192W,S. cerevisiae,gene is orthologous to YGL192W in S. cerevisia...


In [203]:

aerts_tbl.to_csv("motifs-v10-zfish_updated.tbl",index=False,sep ='\t')


from pycistarget.utils import load_motif_annotations

motifs_ort=load_motif_annotations(specie = None, fname = "motifs-v10-zfish_updated.tbl")


In [204]:
motifs_ort[motifs_ort.Motif_similarity_and_Orthology_annot.str.contains(r"MET",na=False,case=False)]



,Direct_annot,Motif_similarity_annot,Orthology_annot,Motif_similarity_and_Orthology_annot
MotifID,,,,
cisbp__M00313,pitx2,"pitx1, pitx2, crx, pou2f1b, otx5, pou2f1a",NaN,"otx1, bmal1a, pitx3, dmbx1b, gsc, dmbx1a, met,..."
cisbp__M00314,pitx2,"pitx1, pitx2",NaN,"otx1, crx, bmal1a, pitx3, dmbx1b, gsc, dmbx1a,..."
hocomoco__MLX_HUMAN.H11MO.0.D,mlx,"arnt, bhlhe40, usf1l, usf1, max",NaN,"met, bmal1a, bmal1b"
metacluster_0.2,"otx1, crx, pitx2, rx2, pitx3, dmbx1b, rx3, gsc...","yy1a, hoxa13a, dmbx1b, dmbx1a, purab, gata6, p...","otx1, crx, pitx2, pitx3, dmbx1b, gsc, dmbx1a, ...","otx1, crx, pitx2, bmal1a, pitx3, dmbx1b, gsc, ..."
metacluster_128.1,"clockb, zbtb14, clocka, mycn, max","her12, clockb, her2, figla, mycb, hey2, her4.1...","her12, hes2.1, helt, her2, met, heyl, her3, he...","her12, hes2.1, helt, her2, mycb, met, heyl, he..."
...,...,...,...,...
metacluster_14.4,NaN,"clocka, clockb",bhlhe40,"met, bmal1a, bmal1b"
metacluster_14.7,NaN,"clockb, myca, meis1b, tfap2c, mycb, mlx, arnt,...",NaN,"her12, hes2.1, helt, her2, met, heyl, her3, he..."
metacluster_183.4,NaN,"clockb, erfl3, hoxb2a, figla, gabpa, etv5a, ho...",ets1,"etv7, elf2b, bmal1a, etv6, elf3, met, elf1, bm..."


In [59]:

##############################################################################################################################################
ens_human_zf_mapping.iloc

In [60]:
#combine orthologies
gene_orthologies=pd.concat([ens_human_zf_mapping.iloc[:,[2,4]],alliance_human_to_zfish.iloc[:,[1,5]],oma_human_to_zfish.iloc[:,[15,19]]])
#gene_orthologies=pd.concat([ens_human_zf_mapping.iloc[:,[2,4]],alliance_human_to_zfish.iloc[:,[1,5]]])
gene_orthologies


,hg_symbol,zf_symbol
13,AACS,aacs
24,AAMDC,aamdc
25,AAMP,aamp
27,AAR2,aar2
32,AARSD1,aarsd1
...,...,...
85209,TRIM32,trim32
85210,STYXL2,dusp27
85213,CREG1,creg1
85218,LEPROTL1,zgc:92045


In [61]:
aerts_tbl=aerts_all_tbl.merge(gene_orthologies,left_on=aerts_all_tbl["gene_name"].str.lower(),right_on=gene_orthologies['hg_symbol'].str.lower())
aerts_tbl

aerts_tbl['gene_name']=aerts_tbl['zf_symbol']
aerts_tbl=aerts_tbl.drop(columns=['hg_symbol', 'zf_symbol'])

aerts_tbl

aerts_tbl.to_csv("motifs-v10-zfish_updated_allspecies.tbl",index=False,sep ='\t')



,Direct_annot,Motif_similarity_annot,Orthology_annot,Motif_similarity_and_Orthology_annot
MotifID,,,,
cisbp__M00313,pitx2,"pou2f1a, pitx1, pou2f1b, crx, pitx2, otx5",NaN,"pitx3, otx2a, bmal1b, dmbx1b, otx1, otx2b, bma..."
cisbp__M00314,pitx2,"pitx2, pitx1",NaN,"pitx3, otx2a, bmal1b, dmbx1b, otx1, otx2b, bma..."
hocomoco__MLX_HUMAN.H11MO.0.D,mlx,"usf1l, bhlhe40, usf1, max, arnt",NaN,"bmal1b, bmal1a, met"
metacluster_0.2,"pitx3, otx2a, dmbx1b, pitx1, rx2, otx1, otx2b,...","otx2a, otx1, tcf7l2, e2f1, ikzf2, hoxa13a, pdx...","pitx3, otx2a, dmbx1b, pitx1, otx1, otx2b, crx,...","pitx3, otx2a, bmal1b, dmbx1b, pitx1, otx1, otx..."
metacluster_128.1,"clocka, max, clockb, mycn, zbtb14","her15.1, her15.2, tfe3a, hey2, tfap4, figla, c...","her15.1, her15.2, hey2, bmal1b, bhlhe40, her12...","her15.1, her15.2, hey2, bmal1b, bhlhe40, her12..."
...,...,...,...,...
metacluster_14.4,NaN,"clockb, clocka",bhlhe40,"bmal1b, bmal1a, met"
metacluster_14.7,NaN,"meis1a, usf2, usf1l, clocka, bhlhe40, meis1b, ...",NaN,"her15.1, her15.2, hey2, bmal1b, bhlhe40, her12..."
metacluster_183.4,NaN,"etv5b, gabpa, figla, clocka, etv1, erfl3, cloc...",ets1,"elf1, bmal1b, elf2b, elf3, elf2a, met, etv7, e..."


In [63]:

from pycistarget.utils import load_motif_annotations

motifs_ort=load_motif_annotations(specie = None, fname = "motifs-v10-zfish_updated_allspecies.tbl")
motifs_ort[motifs_ort.Motif_similarity_and_Orthology_annot.str.contains(r"MET",na=False,case=False)]


,Direct_annot,Motif_similarity_annot,Orthology_annot,Motif_similarity_and_Orthology_annot
MotifID,,,,
cisbp__M00313,pitx2,"pitx3, otx2a, dmbx1b, pou2f1a, pitx1, otx1, ot...",NaN,"pitx3, otx2a, bmal1b, dmbx1b, pou2f1a, pitx1, ..."
cisbp__M00314,pitx2,"pitx3, otx2a, dmbx1b, pitx1, otx1, otx2b, crx,...",NaN,"pitx3, otx2a, bmal1b, dmbx1b, pitx1, otx1, otx..."
cisbp__M00418,otx2b,"pitx3, otx2a, dmbx1b, pou2f1a, pitx1, otx1, cr...",otx2b,"pitx3, otx2a, bmal1b, dmbx1b, pou2f1a, pitx1, ..."
cisbp__M00489,"otx5, crx","pitx3, otx2a, dmbx1b, pou2f1a, pitx1, otx1, ot...","otx5, crx","pitx3, otx2a, bmal1b, dmbx1b, pou2f1a, pitx1, ..."
cisbp__M01737,figla,"mybl1, clocka, gcm2, rfx3, clockb, hoxd12a, te...",figla,"myb, bmal1b, mybl1, clocka, gcm2, rfx3, met, c..."
...,...,...,...,...
metacluster_14.27,NaN,"tfe3a, clocka, bhlhe40, max, clockb, srebf1, b...",NaN,"her15.1, her15.2, mitfb, tfe3a, hey2, bmal1b, ..."
metacluster_14.29,NaN,"bhlhe40, arnt",NaN,"bmal1b, bhlhe40, met, bmal1a, arnt"
metacluster_14.7,NaN,"meis1a, usf2, usf1l, clocka, bhlhe40, meis1b, ...",NaN,"her15.1, her15.2, hey2, bmal1b, clocka, hes6, ..."


In [1]:
import pybiomart as pbm
import pandas as pd

#run_pycistarget(
#    region_sets = region_sets,
#    species = 'homo_sapiens',
#    save_path = os.path.join(outDir, 'motifs'),
#    ctx_db_path = rankings_db,
#    dem_db_path = scores_db,
#    path_to_motif_annotations = motif_annotation,
#    run_without_promoters = True,
#    n_cpu = 8,
#    _temp_dir = os.path.join(tmpDir, 'ray_spill'),
#    annotation_version = 'v10nr_clust',
#    )
dataset = pbm.Dataset(name='drerio_gene_ensembl',  host='http://asia.ensembl.org')

In [5]:
annot = dataset.query(attributes=['chromosome_name', 'transcription_start_site', 'strand', 'external_gene_name', 'transcript_biotype'])
annot


,Chromosome/scaffold name,Transcription start site (TSS),Strand,Gene name,Transcript type
0,24,20927202,1,fam162a,protein_coding
1,24,20927135,1,fam162a,protein_coding
2,17,50472985,-1,si:ch211-235i11.3,protein_coding
3,17,50472864,-1,si:ch211-235i11.3,protein_coding
4,1,49266886,1,caly,protein_coding
...,...,...,...,...,...
61625,11,43070883,1,CABZ01080006.1,lincRNA
61626,11,43643158,1,CABZ01073015.1,lincRNA
61627,11,44120354,-1,FO704721.3,lincRNA
61628,11,44116448,-1,FO704721.3,lincRNA


In [6]:
annot.to_csv(r'SCENIC_MARCH2024/annot1.csv',index=False)


In [7]:
annot = dataset.query(attributes=['chromosome_name', 'start_position', 'end_position', 'strand', 'external_gene_name', 'transcription_start_site', 'transcript_biotype'])


HTTPError: 404 Client Error: Not Found for url: http://asia.ensembl.org:80/biomart/martservice?query=%3CQuery+virtualSchemaName%3D%22default%22+formatter%3D%22TSV%22+header%3D%221%22+uniqueRows%3D%221%22+datasetConfigVersion%3D%220.6%22%3E%3CDataset+name%3D%22drerio_gene_ensembl%22+interface%3D%22default%22%3E%3CAttribute+name%3D%22chromosome_name%22+%2F%3E%3CAttribute+name%3D%22start_position%22+%2F%3E%3CAttribute+name%3D%22end_position%22+%2F%3E%3CAttribute+name%3D%22strand%22+%2F%3E%3CAttribute+name%3D%22external_gene_name%22+%2F%3E%3CAttribute+name%3D%22transcription_start_site%22+%2F%3E%3CAttribute+name%3D%22transcript_biotype%22+%2F%3E%3C%2FDataset%3E%3C%2FQuery%3E